In [ ]:
import numpy as np

from scipy import stats
from scipy import signal
from scipy import integrate
from scipy import fftpack

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from utils import printMatrix

plt.style.use('seaborn-pastel')

## Simulación de edificio de 4 pisos como masas puntuales con rigideces intermedias

In [ ]:
# Edificio de 4 pisos de 20 toneladas cada uno, con rigideces laterales de 57MN/m entre si
k = 57e6 # N/m
m = 20e3 # kg

k1, k2, k3, k4 = k, k, k, k
m1, m2, m3, m4 = m, m, m, m

# Matrices de rigidez y de masa
K = np.array([[k1+k2, -k2, 0, 0],
             [-k2, k2+k3, -k3, 0],
             [0, -k3, k3+k4, -k4],
             [0, 0, -k4, k4]])

M = np.diag((m1, m2, m3, m4))

print(('K = {}').format(K))
print(('M = {}').format(M))

In [ ]:
# Resuelvo autovectores y autovalores
A = np.linalg.inv(M)@K
lamb, eigv = np.linalg.eig(A)

In [ ]:
# convierto velocidades angulares a frecuencias
w_modes = np.sqrt(lamb)
f = w_modes/2/np.pi
print(('f = {}').format(np.round(f,3)))

In [ ]:
# Matriz de autovectores (columnas)
print("eigv:")
printMatrix(eigv)

In [ ]:
# Matriz de masas modales
M_modal = np.transpose(eigv)@M@eigv
M_modal[M_modal<1e-3] = np.nan
print("M_modal:")
printMatrix(M_modal)

In [ ]:
# Matriz de rigideces modales
K_modal = np.transpose(eigv)@K@eigv
K_modal[K_modal<1e-3] = np.nan
print("K_modal:")
printMatrix(K_modal)

In [ ]:
# Verifico la validez de las matrices obteniendo las fercuencias modales nuevamente, ahora como w_i = sqrt(k_i/m_i)
w_2 = np.zeros(len(w_modes))
for mode in range(len(w_2)):
    w_2[mode] = np.sqrt(K_modal[mode,mode]/M_modal[mode,mode])
f_2 = w_2/np.pi/2
print(('f2 = {}').format(f_2))

In [ ]:
# Normalizo vectores modales
eigv_norm = np.zeros(eigv.shape)
for mode in range(len(w_modes)):
    eigv_norm[:,mode] = eigv[:,mode]/max(abs(eigv[:,mode]))
print("eigv_norm:")
printMatrix(eigv_norm)

In [ ]:
fig, ax = plt.subplots()
fig.add_axes()
ax.set(xlim=[-1,1], ylim=[0,5], )
ax.grid()
ax.set_aspect('equal')
eigv_plot = np.zeros((eigv.shape[1] + 1, eigv.shape[1]))
eigv_plot[1:, :] = eigv

ax.plot(np.zeros(eigv.shape[1] + 1),np.linspace(0,4,5), color='k', marker='o')
for col in range(eigv.shape[1]):
    ax.plot(eigv_plot[:,col],[0, 1, 2, 3, 4], marker='o')
    
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.add_axes()
ax.set(xlim=[-1,1], ylim=[0,5], )
ax.grid()
ax.set_aspect('equal')

line1, = ax.plot([], [], lw=3)
line2, = ax.plot([], [], lw=3)
line3, = ax.plot([], [], lw=3)
line4, = ax.plot([], [], lw=3)

def init():
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    line4.set_data([], [])
    return line1, line2, line3, line4

def animate(i):
    y = np.linspace(0, 4, 5)
    line1.set_data(eigv_plot[:,0]*np.sin(0.2*np.pi*i), y)
    line2.set_data(eigv_plot[:,1]*np.sin(0.2*np.pi*i), y)
    line3.set_data(eigv_plot[:,2]*np.sin(0.2*np.pi*i), y)
    line4.set_data(eigv_plot[:,3]*np.sin(0.2*np.pi*i), y)
    return line1, line2, line3, line4

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=10, interval=200, blit=True)

anim.save('sine_wave.gif', writer='imagemagick')

## Simulación de aceleración con perfil de ruido blanco

In [ ]:
# Aceleracion maxima en funcion del tiempo
def max_accel(t,t_end):
    a = 0.5*(1-np.cos(2*np.pi*t/t_end))
    return a

In [ ]:
# Simulacion de aceleracion en forma de ruido blanco bajo la curva de aceleracion maxima
t_end = 600
delta_t = 1/119
t = np.linspace(0, t_end, t_end/delta_t)

n_uni = stats.uniform.rvs(size=len(t))
X = stats.norm.ppf(n_uni) # percentiles para cada valor de n_uni
X_mean = np.mean(X)

accel = 0.1*(X - X_mean)*max_accel(t,t_end) # aceleracion escalada por factor r
# accel = (X - X_mean)
max_accel_vec = max_accel(t,t_end)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
fig.add_axes()
ax.plot(t, accel, color='k', marker='')
ax.plot(t, max_accel_vec, color='r')
ax.grid(True, markevery=1)

In [ ]:
plt.hist(X, bins=100, density=1)
plt.show()

In [ ]:
# PSD de aceleraciones en la base
freq, psd = signal.welch(accel, 
                      fs=1./(delta_t), # sample rate
                      window='hanning', # apply a Hanning window before taking the DFT
                      nperseg=len(t)//27, # compute periodograms of 256-long segments of x
                      detrend='constant') # detrend x by subtracting the mean)
plt.figure()
plt.semilogy(freq[1:], psd[1:])
plt.xlabel('frequency [Hz]')
plt.ylabel('ASD [g²/Hz]')
plt.show()

In [ ]:
print('ASD promedio: {:.3f} g²/Hz'.format(np.mean(psd)))
print('Grms para ASD constante = ASD*f_max = {:.3f} g_rms'.format(np.mean(psd)*f[-1]))

In [ ]:
# calculo de g_rms por integracion
area = integrate.simps(psd, freq)
g_rms = np.sqrt(area)
print('Grms = {:.3f} g_rms'.format(g_rms))

In [ ]:
Ug = np.copy(accel)
fft_freq = fftpack.rfftfreq(len(Ug), d=delta_t)
Ug_fft = abs(fftpack.rfft(Ug))

In [ ]:
plt.figure()
plt.semilogy(fft_freq, Ug_fft)
plt.title('Aceleracion en la base')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Espectro de aceleraciones [g/Hz]')
plt.show()

## Simulación de respuesta dinámica del edificio a ruido blanco su base

In [ ]:
xi = 0.025 # fracción de amortiguamiento crítico
r = np.array([1, 1, 1, 1]).reshape(4,1) # vector logico de desplazamientos respecto de la base
print('xi = {}'.format(xi))
print('r = {}^T'.format(r[:,0]))

La solucion del espectro de desplazamientos para cada modo es
<br/>
<center> $Y(\omega) = \frac{\frac{\iota}{m_i} U_g(\omega)}{\omega_i^2 - \omega^2 + 2i \xi_i \omega_i \omega}$ </center>
<br/>
con
<center> $\iota = \Phi^T M r$. </center>
<br/>
Y la aceleración es
<center> $\ddot{Y}(\omega) = \omega^2 Y(\omega)$ </center>

In [ ]:
#U_g = np.copy(psd)
#w = 2*np.pi*freq
U_g = np.copy(Ug_fft)
w = 2*np.pi*fft_freq

In [ ]:
I = np.transpose(eigv)@M@r
print(I)
m_modal = M_modal.diagonal()
Y = np.zeros((len(w), len(w_modes)), dtype=complex)
ddotY = np.copy(Y)
for mode in range(Y.shape[1]):
    C = I[mode] / (m_modal[mode]*(w_modes[mode]**2 - w**2 + 2*1j*xi*w_modes[mode]*w))
    Y[:,mode] = C*U_g
    #for idx in range(Y.shape[0]):
        #Y[idx, mode] = (I[mode]*U_g[idx]/M_modal[mode,mode]) / (w[mode]**2 - w_psd[idx]**2 + 2*1j*xi*w[mode]*w_psd[idx])
    ddotY[:,mode] = (w**2)*Y[:,mode]

In [ ]:
f = w/2/np.pi
plt.figure()
plt.semilogy(f, abs(Y[:,3]))
plt.semilogy(f, abs(Y[:,2]))
plt.semilogy(f, abs(Y[:,1]))
plt.semilogy(f, abs(Y[:,0]))
plt.title('Desplazamientos')
plt.xlabel('Frequency [Hz]')
plt.ylabel('ASD [m²/Hz]')
plt.show()

In [ ]:
plt.figure()
plt.semilogy(f, abs(ddotY[:,3]))
plt.semilogy(f, abs(ddotY[:,2]))
plt.semilogy(f, abs(ddotY[:,1]))
plt.semilogy(f, abs(ddotY[:,0]))
plt.title('Aceleraciones')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de Aceleraciones [g/Hz]')
plt.show()

In [ ]:
freq_max = [f[i] for i in ddotY.argmax(axis=0)]
print(('freqs from max response = {}').format(np.round(freq_max, 3)))
print('errors in % = {}'.format(np.round(100*(w_modes/2/np.pi - freq_max) / (w_modes/2/np.pi), 2)))